Este notebook implementa un pipeline de preprocesamiento para un producto satelital SPOT 6 (Nivel ORTHO). El código realiza la calibración radiométrica, una corrección atmosférica (usando el método DOS1) y el enmascaramiento de nubes a partir de la máscara oficial del producto, refinando sus bordes mediante dilatación morfológica.

El script está diseñado para procesar una pequeña "ventana" o subconjunto de la imagen, permitiendo realizar pruebas de calidad y desarrollo de forma rápida y con bajo consumo de recursos. El resultado final es un archivo GeoTIFF listo para el análisis (analysis_ready_data), con los valores escalados a UInt16.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rasterio geopandas scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 81.2 MB/s eta 0:00:00


In [3]:
# ===================================================================
# PASO 2: IMPORTAR LIBRERÍAS Y DEFINIR CLASES
# ===================================================================
from abc import ABC, abstractmethod
import xml.etree.ElementTree as ET
from pathlib import Path
import rasterio
from rasterio.windows import Window
from rasterio.merge import merge
from rasterio.features import rasterize
import geopandas as gpd
import numpy as np
import math
from scipy.ndimage import binary_dilation

class Producto_Satelital_Base(ABC):
    def __init__(self, ruta_producto: Path, ruta_salida: Path):
        if not ruta_producto.is_dir():
            raise FileNotFoundError(f"La ruta del producto no existe: {ruta_producto}")
        self.ruta_base = ruta_producto
        self.ruta_salida = ruta_salida
        self.metadatos = {}
        self.ruta_salida_producto = self.ruta_salida / self.ruta_base.name
        self.ruta_salida_producto.mkdir(parents=True, exist_ok=True)
        self._leer_metadatos()

    @abstractmethod
    def _leer_metadatos(self): pass
    @abstractmethod
    def ejecutar_preprocesamiento(self, ventana: Window = None): pass

class ProductoSPOT6(Producto_Satelital_Base):
    def _leer_metadatos(self):
        xml_files = list(self.ruta_base.rglob('DIM_*.XML'))
        if not xml_files:
            raise FileNotFoundError(f"No se encontró el archivo DIM XML principal en {self.ruta_base} o sus subcarpetas.")

        xml_path = xml_files[0]
        tree = ET.parse(xml_path)
        root = tree.getroot()

        self.metadatos['nivel_procesamiento'] = root.find('.//PROCESSING_LEVEL').text
        center_located_values = root.find(".//Located_Geometric_Values[LOCATION_TYPE='Center']")
        self.metadatos['elevacion_solar'] = float(center_located_values.find(".//SUN_ELEVATION").text)
        self.metadatos['ganancias'] = {b.find('BAND_ID').text: float(b.find('GAIN').text) for b in root.findall(".//Band_Radiance")}
        self.metadatos['bias'] = {b.find('BAND_ID').text: float(b.find('BIAS').text) for b in root.findall(".//Band_Radiance")}
        self.metadatos['irradiancias'] = {b.find('BAND_ID').text: float(b.find('VALUE').text) for b in root.findall(".//Band_Solar_Irradiance")}
        cloud_mask_node = root.find(".//MEASURE_NAME[.='Cloud_Cotation (CLD)']/../Quality_Mask/Component/COMPONENT_PATH")
        if cloud_mask_node is not None:
             self.metadatos['mascara_nubes_gml'] = xml_path.parent / cloud_mask_node.attrib['href']
        self.metadatos['band_order'] = [b.find('BAND_ID').text for b in root.findall(".//Raster_Index")]
        self.metadatos['rutas_tifs'] = [xml_path.parent / f.attrib['href'] for f in root.findall(".//Data_File/DATA_FILE_PATH")]
        print(f"Metadatos de {self.ruta_base.name} cargados exitosamente.")

    def ejecutar_preprocesamiento(self, ventana: Window = None):
        print(f"--- Iniciando Pipeline de Preprocesamiento por Bloques para: {self.ruta_base.name} ---")

        vrt_path = self.ruta_salida_producto / 'source.vrt'
        src_files_to_mosaic = [rasterio.open(fp) for fp in self.metadatos['rutas_tifs']]
        merge(src_files_to_mosaic, dst_path=str(vrt_path))

        with rasterio.open(vrt_path) as src:
            # Si no se define una ventana, se procesa todo.
            if ventana is None:
                ventana = Window(0, 0, src.width, src.height)

            # --- NUEVO PASO: GUARDAR EL RECORTE DE LA IMAGEN CRUDA ---
            print("  -> Guardando recorte de la imagen cruda...")
            raw_crop_data = src.read(window=ventana)
            crop_profile = src.profile
            crop_profile.update(height=ventana.height, width=ventana.width, transform=src.window_transform(ventana))
            raw_crop_path = self.ruta_salida_producto / 'imagen_cruda_recortada.tif'
            with rasterio.open(raw_crop_path, 'w', **crop_profile) as dst:
                dst.write(raw_crop_data)
            print(f"     Recorte crudo guardado en: {raw_crop_path.name}")
            # -----------------------------------------------------------

            profile = src.profile
            nodata_value_uint16 = 0
            profile.update(dtype=rasterio.uint16, nodata=nodata_value_uint16, height=ventana.height, width=ventana.width, transform=src.window_transform(ventana))

            suffix = "_SUBSET" if ventana else ""
            path_final = self.ruta_salida_producto / f'analysis_ready_data{suffix}.tif'

            print("  -> Preparando máscara de nubes...")
            ruta_mascara_gml = self.metadatos.get('mascara_nubes_gml')
            if ruta_mascara_gml and ruta_mascara_gml.exists():
                gdf_clouds = gpd.read_file(ruta_mascara_gml)
                cloud_mask_base = rasterize(
                    shapes=gdf_clouds.geometry, out_shape=(src.height, src.width), transform=src.transform,
                    fill=0, all_touched=True, dtype=np.uint8
                ).astype(bool)
                cloud_mask_refinada = binary_dilation(cloud_mask_base, iterations=5)
            else:
                cloud_mask_refinada = np.zeros(src.shape, dtype=bool)
                print("     Advertencia: No se encontró máscara de nubes.")

            with rasterio.open(path_final, 'w', **profile) as dst:
                block_dn = src.read(window=ventana)

                print("     Procesando bloque único...")
                block_rad = np.zeros_like(block_dn, dtype=np.float32)
                for i in range(src.count):
                    band_id = self.metadatos['band_order'][i]
                    gain, bias = self.metadatos['ganancias'][band_id], self.metadatos['bias'][band_id]
                    block_rad[i] = block_dn[i].astype(np.float32) / gain + bias

                sun_zenith = 90.0 - self.metadatos['elevacion_solar']
                block_ref = np.zeros_like(block_rad, dtype=np.float32)
                for i in range(src.count):
                    band_id = self.metadatos['band_order'][i]
                    esun = self.metadatos['irradiancias'][band_id]
                    dark_object = np.percentile(block_rad[i][block_rad[i] > 0], 1) if np.any(block_rad[i] > 0) else 0
                    path_radiance = dark_object - 0.01 * (esun * math.cos(math.radians(sun_zenith))**2) / (math.pi * 1.0**2)
                    numerator = math.pi * (block_rad[i] - path_radiance) * 1.0**2
                    denominator = esun * math.cos(math.radians(sun_zenith))
                    block_ref[i] = numerator / denominator

                block_mask = cloud_mask_refinada[ventana.row_off:ventana.row_off+ventana.height, ventana.col_off:ventana.col_off+ventana.width]
                block_ref_no_nan = np.where(np.stack([block_mask]*src.count), 0.0, block_ref)
                block_scaled = (np.clip(block_ref_no_nan, 0, 1) * 10000).astype(rasterio.uint16)

                dst.write(block_scaled)

        print(f"--- Pipeline Finalizado. Producto final guardado en: {path_final} ---")
        return path_final


In [ ]:
# ===================================================================
# PASO 3: CONFIGURAR RUTAS Y EJECUTAR
# ===================================================================
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # --- CONFIGURACIÓN DE RUTAS ---
    RUTA_BASE = Path('/content/drive/MyDrive/Colab Notebooks')
    RUTA_DATOS_RAW = RUTA_BASE / "data" / "raw"
    RUTA_DATOS_PROCESADOS = RUTA_BASE / "data" / "processed"

    nombre_carpeta_imagen = "CO_2507090419315"
    ruta_completa_imagen = RUTA_DATOS_RAW / nombre_carpeta_imagen

    # --- DEFINICIÓN DE LA VENTANA DE PRUEBA ---
    ventana_de_prueba = Window(6913, 7539, 1024, 1024)

    # --- EJECUCIÓN ---
    producto_spot = ProductoSPOT6(
        ruta_producto=ruta_completa_imagen,
        ruta_salida=RUTA_DATOS_PROCESADOS
    )
    producto_spot.ejecutar_preprocesamiento(ventana=ventana_de_prueba)

except Exception as e:
    print(f"ERROR: Ha ocurrido un problema.")
    print(f"Detalle: {e}")

Mounted at /content/drive
Metadatos de CO_2507090419315 cargados exitosamente.
--- Iniciando Pipeline de Preprocesamiento por Bloques para: CO_2507090419315 ---
  -> Guardando recorte de la imagen cruda...
     Recorte crudo guardado en: imagen_cruda_recortada.tif
  -> Preparando máscara de nubes...
     Procesando bloque único...
--- Pipeline Finalizado. Producto final guardado en: /content/drive/MyDrive/Colab Notebooks/data/processed/CO_2507090419315/analysis_ready_data_SUBSET.tif ---
